In [2]:
#connect to snowflake
import snowflake.connector

conn = snowflake.connector.connect(
    user='MIH039',
    password='TayTayhmy13!',
    account='okalwbn-ewb46701',
    )

cs = conn.cursor()

In [3]:
# test connection
cs.execute("CREATE WAREHOUSE IF NOT EXISTS my_first_warehouse")

cs.execute("CREATE DATABASE IF NOT EXISTS testdb")

cs.execute("USE DATABASE testdb")

cs.execute("CREATE SCHEMA IF NOT EXISTS testschema")

cs.execute(
    "CREATE OR REPLACE TABLE "    
    "test_table(col1 integer, col2 string)"
    )

cs.execute(
    "INSERT INTO test_table(col1, col2) "
        "VALUES (123, 'test string1'), (456, 'test string2')"
    )

In [4]:
# fetch data
cs.execute('SELECT * FROM test_table')

print(cs.fetchmany(2))

[(123, 'test string1'), (456, 'test string2')]


In [5]:
#import csv files
import os

#get list of csv files in directory
directory_path = '/home/jovyan/Desktop/Data/MonthlyPOData'

csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

In [6]:
#create table
cs.execute("create or replace database CASE464")

In [7]:
cs.execute("use database CASE464")

#create table
cs.execute("""
    CREATE OR REPLACE TABLE purchaseorderheader (
        PurchaseOrderID INT,
        SupplierID INT,
        OrderDate STRING,
        DeliveryMethodID INT,
        ContactPersonID INT,
        ExpectedDeliveryDate STRING,
        SupplierReference VARCHAR(255),
        IsOrderFinalized BOOLEAN,
        Comments TEXT, 
        InternalComments TEXT, 
        LastEditedBy INT,
        LastEditedWhen STRING, 
        PurchaseOrderLineID INT,
        StockItemID INT,
        OrderedOuters INT,
        Description TEXT, 
        ReceivedOuters INT,
        PackageTypeID INT,
        ExpectedUnitPricePerOuter DECIMAL(10, 2), 
        LastReceiptDate STRING,
        IsOrderLineFinalized BOOLEAN, 
        Right_LastEditedBy INT,
        Right_LastEditedWhen STRING 
);
""")

In [8]:
# create stage
cs.execute("CREATE OR REPLACE STAGE my_csv_stage;")

# put files into stage
for file in csv_files:
    file_path = os.path.join(directory_path, file) 
    cs.execute(f"PUT file://{file_path} @my_csv_stage auto_compress=true;")

# copy files into table   
for file in csv_files:
    cs.execute(f"""
        COPY INTO purchaseorderheader
        FROM @my_csv_stage/{file}.gz
        FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY='"' SKIP_HEADER=1);
    """)

In [9]:
#Question 2

cs.execute("""
    select purchaseorderid, sum(ReceivedOuters * ExpectedUnitPricePerOuter) AS POAmount
            from purchaseorderheader
            group by purchaseorderid
            order by purchaseorderid
""")

print(cs.fetchmany(5))

[(1, Decimal('313.50')), (2, Decimal('21732.00')), (3, Decimal('2740.50')), (4, Decimal('42481.20')), (5, Decimal('35067.50'))]


In [10]:
#import XML file
cs.execute("use database CASE464")

#create table
cs.execute("""
    CREATE OR REPLACE TABLE suppliertransactions (
    SupplierTransactionID STRING,
    SupplierID STRING,
    TransactionTypeID STRING,
    PurchaseOrderID STRING,
    PaymentMethodID STRING,
    SupplierInvoiceNumber STRING,
    TransactionDate STRING,
    AmountExcludingTax STRING,
    TaxAmount STRING,
    TransactionAmount STRING,
    OutstandingBalance STRING,
    FinalizationDate STRING,
    IsFinalized STRING,
    LastEditedBy STRING,
    LastEditedWhen STRING
);
""")

In [11]:
import xml.etree.ElementTree as ET

# Load and parse the XML file
tree = ET.parse('/home/jovyan/Desktop/Data/SupplierTransactionsXML.xml')
root = tree.getroot()

# Prepare data for insertion
data = []
for row in root:
    entry = {}
    for child in row:
        entry[child.tag] = child.text
    data.append(entry)

In [12]:
# Insert data into the table
insert_sql = """
INSERT INTO suppliertransactions (
    SupplierTransactionID, SupplierID, TransactionTypeID, PurchaseOrderID, PaymentMethodID,
    SupplierInvoiceNumber, TransactionDate, AmountExcludingTax, TaxAmount, TransactionAmount,
    OutstandingBalance, FinalizationDate, IsFinalized, LastEditedBy, LastEditedWhen
) VALUES (%(SupplierTransactionID)s, %(SupplierID)s, %(TransactionTypeID)s, %(PurchaseOrderID)s,
            %(PaymentMethodID)s, %(SupplierInvoiceNumber)s, %(TransactionDate)s, %(AmountExcludingTax)s,
            %(TaxAmount)s, %(TransactionAmount)s, %(OutstandingBalance)s, %(FinalizationDate)s,
            %(IsFinalized)s, %(LastEditedBy)s, %(LastEditedWhen)s);
"""

# Use executemany to insert the data
cs.executemany(insert_sql, data)

# Commit the transaction
conn.commit()

In [13]:
#Question 4
cs.execute("""
    with 
        POH as (
            select purchaseorderid, sum(ReceivedOuters * ExpectedUnitPricePerOuter) AS POAmount
            from purchaseorderheader
            group by purchaseorderid
            order by purchaseorderid
        ),
        invoice as (
            select purchaseorderid, amountexcludingtax
            from suppliertransactions
        )
    select A.purchaseorderid, round(A.POAmount-B.amountexcludingtax,2) as invoiced_vs_quoted
    from POH A
    join invoice B using(purchaseorderid)
    order by A.purchaseorderid
""")

print(cs.fetchmany(5))

[(1, Decimal('0.00')), (2, Decimal('0.00')), (3, Decimal('0.00')), (4, Decimal('0.00')), (5, Decimal('0.00'))]


In [34]:
#Question 5
cs.execute("use database CASE464")
cs.execute("""
    create or replace view purchase_orders_and_invoices as (
        with 
            orderheader as (
                select purchaseorderid, sum(ReceivedOuters * ExpectedUnitPricePerOuter) AS POAmount, LASTRECEIPTDATE as Date
                from purchaseorderheader
                group by purchaseorderid, LASTRECEIPTDATE
                order by purchaseorderid
            ),
            invoice as (
                select purchaseorderid, amountexcludingtax
                from suppliertransactions
            )
        select A.purchaseorderid, round(A.POAmount-B.amountexcludingtax,2) as invoiced_vs_quoted, A.Date
        from orderheader A
        join invoice B using(purchaseorderid)
        order by A.purchaseorderid
)
""")

print(cs.fetchmany(5))

[('View PURCHASE_ORDERS_AND_INVOICES successfully created.',)]


In [14]:
#import postgres data
import psycopg2

In [15]:
#connect to postgres
conn = psycopg2.connect(
    dbname='WestCoastImporters',
    user='jovyan',
    password='postgres',
    host='127.0.0.1',
    port='8765'
)

cur = conn.cursor()

# create path to csv file
csv_file_path = '/home/jovyan/Desktop/Data/supplier_case.csv'  

#convert data to csv
with open(csv_file_path, 'w') as f:
    cur.copy_expert("COPY supplier_case TO STDOUT WITH CSV HEADER", f)


In [16]:
#create a function to create table
def create_table_sql(csv_file_path, table_name):
    with open(csv_file_path, 'r') as f:
        header = f.readline().strip().split(',')
    
    create_table_sql = f"CREATE OR REPLACE TABLE {table_name} (\n"
    for column in header:
        create_table_sql += f'  "{column}" STRING,\n'
    create_table_sql = create_table_sql.rstrip(',\n') + "\n);"

    return create_table_sql

# create table
table_name = 'supplier_case'
create_table_sql = create_table_sql(csv_file_path, table_name)
print(create_table_sql)

CREATE OR REPLACE TABLE supplier_case (
  "supplierid" STRING,
  "suppliername" STRING,
  "suppliercategoryid" STRING,
  "primarycontactpersonid" STRING,
  "alternatecontactpersonid" STRING,
  "deliverymethodid" STRING,
  "postalcityid" STRING,
  "supplierreference" STRING,
  "bankaccountname" STRING,
  "bankaccountbranch" STRING,
  "bankaccountcode" STRING,
  "bankaccountnumber" STRING,
  "bankinternationalcode" STRING,
  "paymentdays" STRING,
  "internalcomments" STRING,
  "phonenumber" STRING,
  "faxnumber" STRING,
  "websiteurl" STRING,
  "deliveryaddressline1" STRING,
  "deliveryaddressline2" STRING,
  "deliverypostalcode" STRING,
  "deliverylocation" STRING,
  "postaladdressline1" STRING,
  "postaladdressline2" STRING,
  "postalpostalcode" STRING,
  "lasteditedby" STRING,
  "validfrom" STRING,
  "validto" STRING
);


In [17]:
#re-define the short for conn.cursor
cs = conn.cursor()

In [20]:
#connect to snowflake
import snowflake.connector

conn = snowflake.connector.connect(
    user='MIH039',
    password='TayTayhmy13!',
    account='okalwbn-ewb46701',
    )

cs = conn.cursor()

In [36]:
#create stage
cs.execute("USE DATABASE CASE464;")
cs.execute("CREATE OR REPLACE STAGE my_another_csv_stage;")

In [22]:
#load csv file into stage
cs.execute(f"PUT file://{csv_file_path} @my_csv_stage auto_compress=true;")

In [23]:
#create table
cs.execute(f"""
        {create_table_sql}
        """)

In [24]:
#copy supplier_case data into table
copy_sql = f"""
COPY INTO {table_name}
FROM @my_csv_stage/{csv_file_path.split('/')[-1]}.gz
FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER = 1)
ON_ERROR = 'CONTINUE';
"""
cs.execute(copy_sql)

In [25]:
#import txt data
cs.execute("use database CASE464")
cs.execute("create or replace stage my_txt_stage;")

In [26]:
# create path to txt file
txt_path = '/home/jovyan/Desktop/Data/2021_Gaz_zcta_national.txt'

# put txt file into stage
cs.execute(f"PUT file://{txt_path} @my_txt_stage AUTO_COMPRESS=TRUE")

In [27]:
# create table
cs.execute("""
    CREATE OR REPLACE TABLE zipcode_mapping (
        GEOID STRING,
        ALAND INT,
        AWATER INT,
        ALAND_SQMI FLOAT,
        AWATER_SQMI FLOAT,
        INTPTLAT FLOAT,
        INTPTLONG FLOAT
    );
""")

In [28]:
# copy txt file into table
cs.execute("""
    COPY INTO zipcode_mapping
    FROM @my_txt_stage
    FILE_FORMAT = (TYPE = 'CSV' FIELD_DELIMITER = '\t' SKIP_HEADER = 1);
""")

In [29]:
#Question 7 Part A
# create table with unique zipcode in supplier_case
cs.execute("""
        create or replace table unique_zipcode as(
            select distinct "postalpostalcode" as zipcode
            from supplier_case
        );
        """)

In [30]:
#create table matching unique zipcode with zipcode_mapping
cs.execute("""
        create or replace table supplier_stations as(
            select A.*
            from zipcode_mapping A
            join unique_zipcode B on A.GEOID = B.zipcode::numeric
            order by A.GEOID
        );
        """)

In [41]:
#create table matching top 3 nearest stations
cs.execute("""
    create or replace table MatchingStations as(
        
        SELECT 
        A.*,
        B.noaa_weather_station_id AS StationID,
        B.noaa_weather_station_name AS StationName,
        B.country_name,
        B.zip_name,
        st_distance(st_makepoint(A.INTPTLONG, A.INTPTLAT), st_makepoint(B.LONGITUDE, B.LATITUDE)) AS Distance,
        ROW_NUMBER() OVER (PARTITION BY A.GEOID ORDER BY st_distance(st_makepoint(A.INTPTLONG, A.INTPTLAT), st_makepoint(B.LONGITUDE, B.LATITUDE))) AS RowNum
        FROM CASE464.public.supplier_stations A
        JOIN WEATHER__ENVIRONMENT.cybersyn.NOAA_WEATHER_STATION_INDEX B
        ON st_distance(st_makepoint(A.INTPTLONG, A.INTPTLAT), st_makepoint(B.LONGITUDE, B.LATITUDE)) < 500000
        QUALIFY ROW_NUMBER() OVER (PARTITION BY A.GEOID ORDER BY Distance) in (1,2,3)
        ORDER BY A.GEOID
        );
""")

In [42]:
#Question 7 Part B

#create view supplier_zip_code_weather, which contains the nearest stations(in top 3) that have high temperature records
cs.execute("""
    CREATE OR REPLACE VIEW CASE464.public.supplier_zip_code_weather AS(
        SELECT 
            A.zip_name, 
            min(A.RowNum) over (partition by A.zip_name) as RowNum,
            B.DATE, 
            B.value as HighTemp
        FROM CASE464.public.matchingstations A
        JOIN weather__environment.CYBERSYN.NOAA_WEATHER_METRICS_TIMESERIES B
        ON A.stationid = B.NOAA_WEATHER_STATION_ID
        WHERE B.variable = 'maximum_temperature'
        order by A.zip_name, B.DATE
    );
""")

In [43]:
#Question 8
#join supplier_case, supplier_zip_code_weather, and purchase_orders_and_invoices
cs.execute("""
    create or replace view supplier_weather as(
        select A."supplierid", A."suppliername", B.DATE, B.HighTemp, C.purchaseorderid, C.invoiced_vs_quoted
        from supplier_case A
        join supplier_zip_code_weather B on A."postalpostalcode" = B.zip_name
        join purchase_orders_and_invoices C on B.DATE::date = C.Date::date
        order by A."supplierid", B.DATE, C.purchaseorderid
        );
""")
